In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transliterate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.5 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
# import category_encoders as ce
import numpy as np
from sklearn.compose import ColumnTransformer
import torchvision.transforms.v2 as T
import torchvision.models as models
from torch.utils.data import Dataset
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import time
import sympy
import pandas as pd

In [ ]:
import pickle
from catboost import CatBoostClassifier, Pool
#import optuna
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import accuracy_score, recall_score, precision_score,f1_score


In [ ]:
data2 = pd.read_excel('/content/drive/MyDrive/ранжирование мероприятий/Сводный перечень (1).xlsx', sheet_name='Датасет')

удаляю нулевую строку

In [ ]:
data2 = data2.drop([0])

Удален "Суммарный балл по мероприятию" потому что не нужен

In [ ]:
data2 = data2.drop(['Суммарный балл по мероприятию'], axis=1)
data2 = data2.fillna(0)
data2 = data2.astype({'СРОК РЕАЛИЗАЦИИ (начало)': 'int',
                'СРОК РЕАЛИЗАЦИИ (окончание)': 'int',
                  'Профинансировано': int})

<ipython-input-13-8392742b5bcf>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data2 = data2.fillna(0)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
data2.iloc[:, 5:-1 ] = scaler.fit_transform(data2.iloc[:, 5:-1 ])

<ipython-input-16-1cfc0c288ece>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 1. 1. ... 0. 0. 0.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data2.iloc[:, 5:-1 ] = scaler.fit_transform(data2.iloc[:, 5:-1 ])


In [ ]:
data2['Показатели и единицы измерения  (Численность населения, для которого созданы комфортные условия проживания вблизи водных объектов, чел.)'] = np.log1p(data2['Показатели и единицы измерения  (Численность населения, для которого созданы комфортные условия проживания вблизи водных объектов, чел.)'])

In [ ]:
data2 = data2.drop(columns = ['Наименование мероприятия'])

In [ ]:
X2 = data2.drop('Профинансировано', axis=1)
y2 = data2['Профинансировано']

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X2, y2, test_size=0.3, stratify=y2, random_state=42)

In [ ]:
cat_features2 = X2.select_dtypes(include=['object']).columns.tolist()
scale_pos_weight2 = (y2 == 0).sum() / (y2 == 1).sum()

model2 = CatBoostClassifier(
    iterations=500,  # Количество деревьев
    learning_rate=0.05,  # Скорость обучения
    depth=6,  # Глубина деревьев
    loss_function='Logloss',  # Для бинарной классификации
    eval_metric='AUC',  # Оцениваем AUC
    verbose=100,  # Вывод логов каждые 100 итераций
    l2_leaf_reg=3,  # L2 регуляризация
    random_strength = 1,  # Добавляет случайность в процесс построения деревьев
    bagging_temperature=1,  # Добавляет случайность при выборке подвыборок
    scale_pos_weight=scale_pos_weight2,  # Учитываем дисбаланс
    cat_features=cat_features2,  # Категориальные признаки
)

# Обучаем модель
model2.fit(X2, y2, early_stopping_rounds=15)

0:	total: 53.6ms	remaining: 26.8s
100:	total: 492ms	remaining: 1.94s
200:	total: 987ms	remaining: 1.47s
300:	total: 1.42s	remaining: 936ms
400:	total: 1.86s	remaining: 458ms
499:	total: 2.26s	remaining: 0us


In [ ]:
def check_on_all(df, model):
  X_new = data2.drop('Профинансировано', axis=1)
  y_new = data2['Профинансировано']


  y_new_pred = model.predict(X_new)

  # print(classification_report(y_new, y_new_pred))
  Accuracy_1 = accuracy_score(y_new, y_new_pred)
  recall_score_1 = recall_score(y_new, y_new_pred)
  precision_score_1 = precision_score(y_new, y_new_pred)
  f1_score_1 = f1_score(y_new, y_new_pred)
  data__RandomForestClassifier_test = pd.DataFrame({'Метрика' : ['Accuracy', 'Recall_score', 'Precision_score', 'F1_score'], 'CatBoost' : [Accuracy_1, recall_score_1, precision_score_1, f1_score_1]})
  print(data__RandomForestClassifier_test)
  df['Предсказанные значения МО'] = y_new
  df['Предсказанные значения МО'].replace({0: 'нет', 1: 'да'})
  return df

In [ ]:
df = check_on_all(data2, model2)

           Метрика  CatBoost
0         Accuracy       1.0
1     Recall_score       1.0
2  Precision_score       1.0
3         F1_score       1.0


In [ ]:
df['Предсказанные значения МО'].value_counts()

,count
Предсказанные значения МО,
0,1106
1,181


In [ ]:
filename = 'catboost_best.pkl'
with open(filename, 'wb') as file:
    pickle.dump(model2, file)